In [1]:
import numpy as np
import tensorflow as tf
import pyreadr
import pandas as pd
import keras
from keras.layers import Dense,Dropout,BatchNormalization
from keras.models import Sequential,Model
from keras.callbacks import ModelCheckpoint,EarlyStopping,ReduceLROnPlateau
from keras.optimizers import Adam
from keras.regularizers import l1
from sklearn.preprocessing import StandardScaler



/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

In [2]:
from keras.backend.tensorflow_backend import set_session
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True  # dynamically grow the memory used on the GPU
# config.log_device_placement = True  # to log device placement (on which device the operation ran)
sess = tf.Session(config=config)
set_session(sess)  # set this TensorFlow session as the default session for Keras

In [3]:
# Set seeds for random number generators for reproducable results
seed = 0
np.random.seed(seed)
tf.set_random_seed(seed)

In [4]:
# Load training data
data = pyreadr.read_r("/home/shanmukh/Documents/IICT/tep-fault-detection/dataset/TEP_FaultFree_Training.RData")
df = data['fault_free_training']
training_data = df.drop(["faultNumber","simulationRun","sample"],axis=1)

In [5]:
# Standard Normalization
# 0 mean 
# 1 std
scaler = StandardScaler()
scaler.fit(training_data)
training_data = scaler.transform(training_data)

In [6]:
# Get Autoencoder model
def autoencoder(dropout=0.0,regularization = 0.001):
    model = Sequential()
    model.add(Dense(128,input_dim=52,activation="elu"))
    model.add(BatchNormalization())
    model.add(Dropout(dropout))
    model.add(Dense(64,activation="elu"))
    model.add(BatchNormalization())
    model.add(Dropout(dropout))
    model.add(Dense(32, name='latent_space'))
    
    model.add(Dense(64,activation="elu"))
    model.add(BatchNormalization())
#     model.add(Dropout(dropout))
    model.add(Dense(128,activation="elu"))
    model.add(BatchNormalization())
#     model.add(Dropout(dropout))
    model.add(Dense(52))
#     model.summary()
    return model

In [7]:
model = autoencoder()
model.compile(optimizer=Adam(lr=0.001),loss="mse",metrics=['mse'])
# Train Network
callbacks = [ReduceLROnPlateau(monitor='val_loss',factor=0.2,patience=5,verbose=True),
            ModelCheckpoint(filepath='/home/shanmukh/Documents/IICT/tep-fault-detection/models/decoder_no_dropout/weights-{epoch:02d}-{val_loss:.2f}.hdf5',monitor='val_loss',save_best_only=True,verbose=True),
            EarlyStopping(patience=13,restore_best_weights=True)]
model.fit(training_data,training_data,batch_size=256,validation_split=0.2,epochs=1000,callbacks=callbacks)
# Get outputs
# SPE statistic
# Hotelling's T^2 Statistic
# Percentile Tresholds
# Unified Statistic
# Load Testing Data
# Metrics







Train on 200000 samples, validate on 50000 samples
Epoch 1/1000
200000/200000 [==============================] - 15s 75us/step - loss: 0.1986 - mean_squared_error: 0.1986 - val_loss: 0.1083 - val_mean_squared_error: 0.1083

Epoch 00001: val_loss improved from inf to 0.10826, saving model to /home/shanmukh/Documents/IICT/tep-fault-detection/models/decoder_no_dropout/weights-01-0.11.hdf5
Epoch 2/1000
200000/200000 [==============================] - 13s 65us/step - loss: 0.0995 - mean_squared_error: 0.0995 - val_loss: 0.0912 - val_mean_squared_error: 0.0912

Epoch 00002: val_loss improved from 0.10826 to 0.09121, saving model to /home/shanmukh/Documents/IICT/tep-fault-detection/models/decoder_no_dropout/weights-02-0.09.hdf5
Epoch 3/1000
200000/200000 [==============================] - 13s 65us/step - loss: 0.0970 - mean_squared_error: 0.0970 - val_loss: 0.0899 - val_mean_squared_error: 0.0899

Epoch 00003: val_loss improved from 0.09121 to 0.08985, saving model to /home/shanmukh/Doc